# target encoder & ohe & CatBoost

In [2]:
%load_ext autoreload
%autoreload 2

import os
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append(os.path.abspath('..'))
# ---------------------------------
import numpy as np
import pandas as pd
import scipy

import catboost as cb
from category_encoders import TargetEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split 
# ---------------------------------
# from tools import Tuning

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
os.mkdir('kkk') if 

module

In [2]:
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')

# ord_5
for i in range(2):
    train_df[f'ord_5_{i}'] = train_df['ord_5'].str[i]
    test_df[f'ord_5_{i}'] = test_df['ord_5'].str[i]

# fillna
for col in test_df.columns:
    train_df[col].fillna(train_df[col].mode()[0], inplace=True)
    test_df[col].fillna(test_df[col].mode()[0], inplace=True)

# target
target = train_df['target']
y_train = target.values

# drop
train_df.drop(['target', 'ord_5', 'id'], axis=1, inplace=True)
test_df.drop(['ord_5', 'id'], axis=1, inplace=True)

In [3]:
train_df = train_df.astype('str')
test_df = test_df.astype('str')

## Tuning

In [4]:
def neg_auc(y_true, y_pred):
    return - roc_auc_score(y_true, y_pred)

model = cb.CatBoostClassifier

model_fix_params = {
    'eval_metric': 'AUC', 
    'iterations': 10000,
    'learning_rate': 0.1, 
    'random_seed':233333,
    'task_type':'GPU', 
    'verbose':0,
    'thread_count': 4,
    'early_stopping_rounds': 500
}

cv_params = {
    'split_method': StratifiedKFold,
    'nflod': 1,
    'shuffle': True,
    'random_state':23333,
    'fit_params':{'cat_features': list(train_df.columns)}
}

In [ ]:
tu = Tuning(train_df, target, model, model_fix_params, cv_params, metrics_func=neg_auc, fit_use_metrics=True)

In [ ]:
sd = {
    'l2_leaf_reg': tu.hp.quniform('l2_leaf_reg', 1, 10, 1),
    'depth': tu.hp.quniform('depth', 4, 10, 1),
    'loss_function': tu.hp.choice('loss_function', ['Logloss', 'CrossEntropy']),
    'bagging_temperature': tu.hp.uniform('bagging_temperature', 0.5, 1),
    'random_strength': tu.hp.uniform('random_strength', 0.5, 1)
}

final = tu.fmin(sd, max_evals=50)

In [ ]:
tu.best_param['learning_rate'] = 0.01
tu.best_param['verbose'] = 1
tu.cv_params['nflod'] = 5
tu.cv_params['fit_params']['verbose_eval'] = 300

y_pred = tu.cv_predict(test_df)

# Submission

In [ ]:
submission = pd.read_csv('../data/sample_submission.csv', index_col='id')
submission['target'] = y_pred
submission.to_csv('logit.csv')